In [2]:
#Importing all Necessary libraries and packages

import nltk
import string
import json

from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Preprocessing data to convert all the sentences into lower case, to remove the non-ascii characters and punctuation

def pre_process(corpus):
    # convert input corpus to lower case.
    corpus = corpus.lower()
    # collecting a list of stop words from nltk and punctuation form
    # string class and create single array.
    stopset = stopwords.words('english') + list(string.punctuation)
    # remove stop words and punctuations from string.
    # word_tokenize is used to tokenize the input corpus in word tokens.
    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    corpus = unidecode(corpus)
    return corpus

In [4]:
#Lemmatizing the data input

lemmatizer = WordNetLemmatizer()

def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:                    
    return None

def lemmatize_sentence(sentence):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))    
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)

  res_words = []
  for word, tag in wn_tagged:
    if tag is None:                        
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))

  return " ".join(res_words)

In [ ]:
with open("TestDf.jsonf.json", 'r') as f:
    datastore = json.load(f)
    
corpus = []
video_id = []

for item in datastore:
    corpus.append(item['caption'])

In [26]:
# sentence pair

for c in range(len(corpus)):
    corpus[c] = pre_process(corpus[c])
    corpus[c] = lemmatize_sentence(corpus[c])
    print(corpus[c])

woman cut end carrot slice
woman cut carrot


In [27]:
# creating vocabulary using uni-gram and bi-gram
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2))
tfidf_vectorizer.fit(corpus)

In [ ]:
captions = ["A woman is cutting an end of carrot into slices.", "A woman is cutting carrot"]

feature_vectors = tfidf_vectorizer.transform(captions)

In [28]:
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

In [29]:
get_cosine_similarity(feature_vectors[0], feature_vectors[1])

0.43927499031635536